# About Pandas

Pandas is one of the most popular tools for data wrangling, preparation, analysis and modelling. It allows python developers to work on various data sources including delimited files, html pages(requires the presence of table tags), json, hdf5 files, sql data from databases etc. This makes it a very versatile tool to working with data.Pandas also offers apis to manipulate data such as filtering, indexing, grouping by and aggregate functions such as mean, standard deviation, kurtosis, etc. It also allows manipulation of data using regular and lambda functions. SQL-like functions make it possible to perform cartesian joins using functions like `join` and `merge`. For data cleaning operations, offers functions for checking for nulls, handling nulls, dropping rows and columns.

## Basic Info

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(
    "movie_data.txt", 
    delimiter = "\t", 
    names = ["ID","title","year",
             "rating","votes",
             "duration","genres"],
    index_col=0)

In [3]:
df.head()

,title,year,rating,votes,duration,genres
ID,,,,,,
tt0111161,The Shawshank Redemption (1994),1994,9.2,619479,142 mins.,Crime|Drama
tt0110912,Pulp Fiction (1994),1994,9.0,490065,154 mins.,Crime|Thriller
tt0137523,Fight Club (1999),1999,8.8,458173,139 mins.,Drama|Mystery|Thriller
tt0133093,The Matrix (1999),1999,8.7,448114,136 mins.,Action|Adventure|Sci-Fi
tt1375666,Inception (2010),2010,8.9,385149,148 mins.,Action|Adventure|Sci-Fi|Thriller


In [4]:
df.tail(11)

,title,year,rating,votes,duration,genres
ID,,,,,,
tt0101356,Another You (1991),1991,4.9,1359,98 mins.,Comedy|Crime
tt0421090,Zerophilia (2005),2005,6.3,1359,90 mins.,Comedy|Romance
tt0067227,The Merchant of Four Seasons (1971),1971,7.6,1359,88 mins.,Drama
tt0339727,Stateside (2004),2004,5.8,1358,97 mins.,Drama|Music|Romance
tt0218581,Scarlet Diva (2000),2000,5.2,1358,91 mins.,Drama
tt0118635,Aprile (1998),1998,6.7,1358,78 mins.,Comedy
tt0807721,Meduzot (2007),2007,7.0,1357,78 mins.,Drama
tt0339642,Daltry Calhoun (2005),2005,5.2,1357,100 mins.,Comedy|Drama|Music|Romance
tt0060880,The Quiller Memorandum (1966),1966,6.5,1356,104 mins.,Drama|Mystery|Thriller


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, tt0111161 to tt0279977
Data columns (total 6 columns):
title       10000 non-null object
year        10000 non-null int64
rating      10000 non-null float64
votes       10000 non-null int64
duration    10000 non-null object
genres      9999 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 546.9+ KB


## Cleaning

In [6]:
df = df.dropna(axis = 1)
print(df.columns)

Index(['title', 'year', 'rating', 'votes', 'duration'], dtype='object')


In [7]:
df.duration = df.duration.apply(lambda dur:int(dur.split(" ")[0])*60)


df.title = df.title.apply(lambda title: title.split("(")[0])
print(df.columns)
df.head()

Index(['title', 'year', 'rating', 'votes', 'duration'], dtype='object')


,title,year,rating,votes,duration
ID,,,,,
tt0111161,The Shawshank Redemption,1994,9.2,619479,8520
tt0110912,Pulp Fiction,1994,9.0,490065,9240
tt0137523,Fight Club,1999,8.8,458173,8340
tt0133093,The Matrix,1999,8.7,448114,8160
tt1375666,Inception,2010,8.9,385149,8880


## Concatenating and Merging Data Frames

### Loading Movie Crew data

In [8]:
movie_crew_df = pd.read_csv("crew_data.tsv",
                            delimiter = "\t",
                           names=["ID","directors","writers"],
                           index_col=0,
                           skiprows=1,
                           usecols=["ID","directors"])
movie_crew_df.columns

FileNotFoundError: [Errno 2] File b'crew_data.tsv' does not exist: b'crew_data.tsv'

In [ ]:
movie_crew_df.head()

### Loading Person Data

In [ ]:
person_df = pd.read_csv("name_data.tsv",
                       delimiter="\t",
                        names=["nconst","primaryName"],
                       usecols=["nconst","primaryName"],
                       skiprows=1)
person_df.columns = ["person_ID","name"]
person_df.set_index = 0

In [ ]:
person_df.head()

### Merging Crew and Movie Data

In [ ]:
df=pd.merge(df, movie_crew_df,left_index=True,right_index=True)
df.head()

#### Dealing with multiple directors

In [ ]:
df = df[df['directors'].str.contains(",")==False]
df.rename(columns={'directors':'director_id'},inplace=True)
df.head()

### Merging with Person Data

In [ ]:
df = pd.merge(df,person_df,
              left_on='director_id',
              right_on='person_ID')
df.head()

In [ ]:
df.rename(columns={'name':'director'},inplace=True)
df.head()

#### Extra clean up

In [ ]:
df = df.drop(columns=["person_ID"])
df.head()

## Data Exploration

### Ten longest movies

In [ ]:
result=df.sort_values(["duration"], ascending=[False])
result.head(10)

### Best rated movies ordered by Rating and Title

In [ ]:
result=df.sort_values(["rating","title"],ascending=[False,True])
result.head(10)

### Average Duration of a Movie

In [ ]:
result = df.duration.mean()
print(result)
print(f"The average duration of a movie is {result/60}")

### Ten most productive directors

In [ ]:
grouped = df.groupby(["director"])[["director"]].count()
grouped.rename(columns={"director":"count"},inplace=True)
grouped.head()

In [9]:
result= grouped.sort_values(["count"],ascending=[False])
result.head(10)

NameError: name 'grouped' is not defined

### How many movies were made in the 2000's

In [ ]:
result=df[(df.year>1999) & (df.year<2010)].year.count()
print(f"The number of movies made in the 2000s is {result}")

### Movies made by Akira Kurosawa ordered by year DESC

In [ ]:
result=df[df.director=="Akira Kurosawa"].sort_values(["year"],ascending=[False])[["title","year","director"]]
result.head(20)